In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from newrow import newrow_cash, newrow_etf
from utils import get_ter_requests, broker_fee, get_date

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("report.csv")

In [31]:
df

,Data,Operazione,SIM,Prodotto,ISIN,Asset Name,TER,Valuta,QT. Scambio,Prezzo,Imp. Nominale Operaz.,Commissioni,QT. Attuale Asset,Imp. Residuo Asset,PMPC Asset,Imp. Effettivo Operaz.,Costo Rilasciato,Plusv. Lorda,Minusv. Generata,Scadenza,Zainetto Fiscale,Plusv. Imponibile,Imposta,Netto,Liquidita Attuale,Valore Titoli,NAV,Liq. Storica Immessa
0,30-05-2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,28-09-2025,deposit,BG Saxo,NaN,NaN,NaN,NaN,EUR,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1000.0,1000.0,0.0,1000.0,1000.0
2,28-09-2025,buy,BG Saxo,ETF,IE00B4L5Y983,ISHARES CORE MSCI WORLD,0.20% p.a.,EUR,10.0,-10.0,-100.0,5.0,10.0,105.0,10.5,-105.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,895.0,100.0,995.0,1000.0
3,28-09-2025,buy,BG Saxo,ETF,IE00B4L5Y983,ISHARES CORE MSCI WORLD,0.20% p.a.,EUR,5.0,-14.0,-70.0,5.0,15.0,180.0,12.0,-75.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,820.0,210.0,1030.0,1000.0
4,28-09-2025,sell,BG Saxo,ETF,IE00B4L5Y983,ISHARES CORE MSCI WORLD,0.20% p.a.,EUR,8.0,9.0,72.0,5.0,7.0,84.0,12.0,67.0,96.0,0.0,29.0,31-12-2029,29.0,0.0,0.0,-29.0,887.0,63.0,950.0,1000.0
5,28-09-2025,sell,BG Saxo,ETF,IE00B4L5Y983,ISHARES CORE MSCI WORLD,0.20% p.a.,EUR,2.0,11.0,22.0,5.0,5.0,60.0,12.0,17.0,24.0,0.0,7.0,31-12-2029,36.0,0.0,0.0,-7.0,904.0,55.0,959.0,1000.0
6,28-09-2025,sell,BG Saxo,ETF,IE00B4L5Y983,ISHARES CORE MSCI WORLD,0.20% p.a.,EUR,1.0,20.0,20.0,5.0,4.0,48.0,12.0,15.0,12.0,3.0,0.0,NaN,33.0,0.0,0.0,3.0,919.0,80.0,999.0,1000.0
7,28-09-2026,sell,BG Saxo,ETF,IE00B4L5Y983,ISHARES CORE MSCI WORLD,0.20% p.a.,EUR,1.0,10.0,10.0,5.0,3.0,36.0,12.0,5.0,12.0,0.0,7.0,31-12-2030,40.0,0.0,0.0,-7.0,924.0,30.0,954.0,1000.0
8,01-02-2030,sell,BG Saxo,ETF,IE00B4L5Y983,ISHARES CORE MSCI WORLD,0.20% p.a.,EUR,1.0,20.0,20.0,5.0,2.0,24.0,12.0,15.0,12.0,3.0,0.0,NaN,4.0,0.0,0.0,3.0,939.0,40.0,979.0,1000.0


### MANUALLY REMOVE LAST ROW:

In [26]:
df = df.iloc[:-1]

### IMPLEMENTATI:

1. ~~Liquidity~~
2. ~~ETF~~
3. `TODO` Stock
4. `TODO` Bond
5. ~~Esci dal programma~~
- `TODO` Gestione separata intermediari 
- `TODO` Conversione valute

In [ ]:
def main_menu():
    print("\n1. Liquidità")
    print("2. ETF")
    print("3. Azioni")
    print("4. Obbligazioni")
    print("5. Esci dal programma")

while True:
    main_menu()
    choice = input("Seleziona operazione (1 2 3 4 5): ")

    # CASH
    if choice == '1':

        date = get_date(df)

        brk = int(input("Seleziona intermediario (SIM) (1. Fineco, 2. BG Saxo): "))
        brokers = {
            1: "Fineco",
            2: "BG Saxo",
        }
        broker = brokers.get(brk, "SIM non riconosciuto")
        
        cash = float(input("Inserisci contante da depositare o prelevare: "))
        if cash == 0.0:
            raise ValueError("Il contante inserito non può essere 0€.\nPositivo se depositato, negativo se prelevato.")
        
        
        df = newrow_cash(df, date, cash, broker)

    
    # ETF
    elif choice == '2':

        date = get_date(df)

        brk = int(input("Enter broker (1. Fineco, 2. BG Saxo): "))
        isin = input("Enter isin: ")

        qt = input("Enter quantity: ")
        if not (qt.isdigit() and int(qt) > 0):
            raise ValueError("Quantit must be a strictly positive integer")
        else:
            quantity = int(qt)


        price = float(input("Enter price: "))
        if price == 0:
            raise ValueError("Price cannot be 0")
        
        broker, fee = broker_fee(brk, "ETF") 

        if price < 0:   # buy
            buy = True
            difference = df["Liquidita Attuale"].iloc[-1] + (quantity * price - fee)
            #if difference < 0:
            #    raise ValueError(f"Insufficient funds: {df["Liquidita Attuale"].iloc[-1]}€ {quantity * price - fee}€ = {difference}€")
        
        else:           # sell
            buy = False
        
        ter, err = get_ter_requests(isin)
        if not ter:
            print(f"TER could not be retrieved for ISIN {isin}")
            ter = input("Manually enter TER: ")

        df = newrow_etf(df, date, isin, quantity, price, ter, broker, fee, buy)

    
    
    elif choice == '3':
        print("Viewing items...")

        
    elif choice == '4':
        print("Goodbye!")
        break
    else:
        print("Invalid choice! Please try again.")



1. Liquidity
2. ETF
3. Stock
4. Exit



1. Liquidity
2. ETF
3. Stock
4. Exit
Goodbye!


In [ ]:
df.to_csv("report.csv", index=False)